# Optimizing your cloud

In this tutorial we will try to optimize our cloud storage, and in the process explore some concepts and methods in the
field of optimization. We will look at brute force methods, simple heuristics, an iterative approach, and model
building.

After your computer got infected with ransomware, and you had to pay a hefty sum of money to send **haxxorKid420** a
weirdly over prized picture of a green frog that apparently resides on the "blockchain", you decide it is time to
finally get around to backing up your files. There is too much hassle with buying hard drives and scheduling automatic
backups, and there is a sale on this new could storage service that spun up last week that you decide to check out. They
have a free plan with up to 1GB of storage, however, you have more than 1GB of data to back up, so what do you do? There
is definitely some data you don't need, so let's try to find out what to keep and not.

You start by cataloging your data, assigning value to the different folders on your computer and calculating their
sizes.

In [ ]:
from selection import Item, Selection

directories = [Item("system files", 423, 5), Item("documents", 57, 7), Item("pictures", 209, 10), Item("music", 114, 5),
               Item("videos", 856, 7), Item("games", 342, 5), Item("talks", 74, 10), Item("image files", 33, 5),
               Item("dotfiles", 2, 8), Item("university notes", 24, 5), Item("Outlook mailboxes", 156, 3),
               Item(".temp", 48, 1), ]

## The brute force approach

Now is the question of how to go about finding the optimal selection of directories that fit on your cloud storage
plan. The simplest is obviously to simply check every combination and select the one that has the highest value.

In [ ]:
from collections.abc import Iterator
from wrappers import timed_method


def enumerate_selections(count: int) -> Iterator[list[bool]]:
    """
    Enumerate all possible selections of a given number of items. Each enumeration is represented as a bit mask where
    a True means the item is included and a False means it isn't.

    Args:
        count: The number of items.

    Returns:
        An iteration of enumerations. Each enumeration has length count.

    """

    def enumerate(items: list[bool], index: int) -> Iterator[list[bool]]:
        """
        Recursive helper function for the method.
        """

        if index == len(items):
            yield items
        else:
            yield from enumerate(items.copy(), index + 1)
            items = items.copy()
            items[index] = True
            yield from enumerate(items, index + 1)

    yield from enumerate([False] * count, 0)


@timed_method
def brute_force(items: list[Item], capacity: float) -> Selection:
    """
    Compute the best selection of files by checking every possible selection and choosing the one with the highest
    score.

    Args:
        items: The items to select from.
        capacity: The capacity of the cloud storage.
    """

    def get_size(selection: list[bool]) -> float:
        """
        Compute the size of the files with the given selection.
        """

        size = 0.0

        for (i, item) in enumerate(items):
            if selection[i]:
                size += item.size

        return size

    def get_score(selection: list[bool]):
        """
        Compute the score of the given selection.
        """

        score = 0.0

        for (i, item) in enumerate(items):
            if selection[i]:
                score += item.value

        return score

    best_score = 0.0
    best_selection: list[bool] = []

    for selection in enumerate_selections(len(items)):

        if (get_size(selection)) > capacity:
            continue

        score = get_score(selection)

        if score > best_score:
            best_score = score
            best_selection = selection

    return Selection.from_mask(items, best_selection)


In [ ]:
solution_brute_force = brute_force(directories, 1024)
print(solution_brute_force)

## A simple (greedy) heuristic

Alright, that worked, and obviously gave us the best solution, but the method isn't very satisfying. There has to be a
better and more efficient way of doing this.

What if we always add the current best single item and keep doing that until we have filled up our storage? This is
what is called a "greedy (construction) heuristic".

❓ _What defines the best item?_

In [ ]:
@timed_method
def greedy_heuristic(items: list[Item], capacity: float) -> Selection:
    """
    Estimate the best selection of files by selecting files with a high value to size ratio until the storage is full.

    Args:
        items: The items to select from.
        capacity: The capacity of the cloud storage.
    """
    selection = [False] * len(items)
    weight = 0.0
    full = False

    while not full:

        best_idx = -1
        best_ratio = 0

        for (i, item) in enumerate(items):
            if selection[i]:
                continue
            if weight + item.size > capacity:
                continue

            ratio = item.value / item.size

            if ratio > best_ratio:
                best_idx = i
                best_ratio = ratio

        if best_idx == -1:
            full = True
        elif all(selection):
            full = True

        if full:
            continue

        selection[best_idx] = 1
        weight += items[best_idx].size

    return Selection.from_mask(items, selection)

In [ ]:
solution_greedy = greedy_heuristic(directories, 1024)
print(solution_greedy)

## Random restart

Even though that gave a pretty good solution, we can see from our brute force attempt that it did not give us the best
solution. However, it gave us a solution in a lot less time. So how about we introduce some randomness to our
construction heuristic and then simply call it multiple times and select the best of them.

❓ _How do we make our greedy heuristic random?_

In [ ]:
import random
from typing import Callable, Any


def greedy_construction_heuristic(items: list[Item], capacity: float) -> Selection:
    """
    A variation of greedy_heuristic that introduces some randomness to the selection process. Instead of simply
    choosing the element with the highest value to size ratio it chooses randomly weighted by that ratio.

    Args:
        items: The items to select from.
        capacity: The capacity of the cloud storage.
    """

    selection = [False] * len(items)
    size = 0.0

    def get_random_item() -> (int, Item):
        """
        Select a random item from the remaining ones that fit on the cloud storage weighted by how good it is.

        Returns:
            The index of the item and the item itself.
        """

        p = [0.0] * len(items)
        has_any = False

        for (index, item) in enumerate(items):
            if selection[index]:
                continue
            if size + item.size > capacity:
                continue

            p[index] = item.value / item.size
            has_any = True

        if not has_any:
            return -1, None

        selected_idx = random.choices(range(len(items)), p, k=1)[0]
        return selected_idx, items[selected_idx]

    while True:

        (index, item) = get_random_item()

        if index == -1:
            break;

        selection[index] = True
        size += item.size

    return Selection.from_mask(items, selection)


Heuristic_Method = Callable[[list[Item], float], Selection]


@timed_method
def random_restart(items: list[Item], capacity: float, iterations: int,
                   heuristic: Heuristic_Method = greedy_construction_heuristic) -> Selection:
    """
    Estimate the best selection of files by randomly trying out different solutions and choosing the best one.

    Args:
        items: The items to select from.
        capacity: The capacity of the cloud storage.
        iterations: The number of random restarts to do.
        heuristic: The construction heuristic to use for every attempt.
    """
    best_score = 0
    best_solution = None

    for itt in range(iterations):

        solution = heuristic(items, capacity)

        if solution.score() > best_score:
            best_score = solution.score()
            best_solution = solution

    return best_solution

In [ ]:
solution_random_restart = random_restart(directories, 1024, 100)
print(solution_random_restart)

## Ruin and recreate

Instead of starting from scratch at every iteration it should also be possible to start from a solution and then change
it by a small amount and see if the solution improves. One such approach is called "ruin and recreate", and as the name
implies it is a simple approach where you first destroy your solution and then rebuild it, hopefully in a way that gives
a different solution to the one you had before.

❓ _What are good ruin and recreate operators?_

In [ ]:
@timed_method
def ruin_and_recreate(items: list[Item], capacity: float, iterations: int):
    """
    Estimate the best selection of files by alternately destroying and fixing an existing solution in the hopes of
    getting a better one.

    Args:
        items: The items to select from.
        capacity: The capacity of the cloud storage.
        iterations: The number of ruin & recreate steps to carry out.
    """

    def rr_ruin(solution: Selection) -> (Selection, list[Item]):
        """
        The ruin operator, removes elements from the provided selection.

        Args:
            solution: The selection to remove elements from

        Returns:
            A copy of the solution with fewer elements and the list of elements that were removed.
        """
        copy = solution.copy()

        removed_items = []
        for i in range(3):
            removed_index = random.randrange(len(copy.items))
            removed_items.append(copy.items.pop(removed_index))

        return copy, removed_items

    def rr_recreate(solution: Selection, removed: list[Item]) -> Selection:
        """
        Fixes a ruined solution by adding random elements back to it.

        Args:
            solution: The ruined solution to fix.
            removed: The elements that were removed by the ruin operator.

        Returns:
            A solution with more elements than the one provided.
        """

        def get_random_item() -> Item:
            """
            Selects a random element not in the selection weighted by the value to size ratio.
            """
            available = []
            for item in items:
                if item in solution.items:
                    continue
                elif item.size + solution.size() > capacity:
                    continue

                available.append(item)

            if len(available) == 0:
                return None

            return random.choices(available, [x.value / x.size for x in available], k=1)[0]

        while True:
            item = get_random_item()

            if item is None:
                break

            solution.items.append(item)

        return solution

    def rr_operation(solution: Selection) -> Selection:
        """
        The ruin then recreate cycle.
        """
        ruined_solution, removed_items = rr_ruin(solution)
        return rr_recreate(ruined_solution, removed_items)

    solution = greedy_construction_heuristic(items, capacity)

    for i in range(iterations):

        test_solution = rr_operation(solution)

        if test_solution.score() > solution.score():
            solution = test_solution

    return test_solution


In [ ]:
solution_ruin_recreate = ruin_and_recreate(directories, 1024, 100)
print(solution_ruin_recreate)

## Selecting files

Things are looking up, and we have multiple ways of finding the best selection of directories to put on our cloud
storage, but why stop there? If we can choose the best folders why not choose the best files instead? Although it is a
bit tedious, you list all the files on your system and gives them a score on how valuable they are to you and store that
in a table. Now it is time to run it through our algorithms and see the optimal selection of files.

In [ ]:
files = Item.from_csv('data/files.csv')
print(f"{len(files)} files, {round(sum([x.size for x in files]))}MB")

In [ ]:
print(f"{float(2 ** len(files))} permutations")
# solution_files_brute_force = brute_force(files, 1024)

In [ ]:
solution_files_greedy = greedy_heuristic(files, 1024)
print(solution_files_greedy)

In [ ]:
solution_files_restart = random_restart(files, 1024, 100)
print(solution_files_restart)

## Modelling and optimzation libraries

As this isn't the first time you have coded something there has been a thought nagging at the back of your mind for the
past couple of minutes, and that is: "there has to be a library for this". After a bit of searching you come across
[pyomo](http://www.pyomo.org) which seems to provide the tools you need. Pyomo is a modelling framework that then sends
your optimization problem to an optimization solver.

Reading the documentation we find out how pyomo works and starts building a model of our optimization problem.

In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, Set, quicksum
import pyomo.environ as pyo

ModelFactory = Callable[[list[Item], int], ConcreteModel]


def create_model(items: list[Item], capacity: int) -> ConcreteModel:
    """
    Create a pyomo model from the list of possible items to put on the cloud.

    Args:
        items: The items to select from.
        capacity: The capacity of the cloud storage.
    """
    model = ConcreteModel()

    # For simplicity, we convert the list of items to a dictionary for easier use
    itemdict = Item.to_dict(items)

    # First define our optimization variables, the variable names are given by the filename, and the variables
    # themselves are binary, either they are included or they are not
    model.filenames = Set(initialize=[item.name for item in items])
    model.files = Var(model.filenames, within=pyo.Binary)

    # The mease of how well we are doing, the objective, is simply the sum of all the values
    model.score = Objective(expr=quicksum([model.files[name] * itemdict[name].value for name in model.filenames]),
                            sense=pyo.maximize)

    # Finally, we define the constraint, that the selection of items have to fit on the cloud
    model.restriction = Constraint(
        expr=quicksum([model.files[name] * itemdict[name].size for name in model.filenames]) <= capacity)

    return model


@timed_method
def solve_pyomo(items: list[Item], capacity: int, solver: str, model_factory: ModelFactory = create_model) -> (
        Selection, ConcreteModel):
    """
    Calculate the best selection of files to put in your cloud storage using the pyomo framework as a solver.

    Args:
        items: The items to select from.
        capacity: The capacity of the cloud storage.
        solver: The solver to use to solve the optimization problem
        model_factory: A method that constructs a mathematical model from our variables

    Returns:
        The solution to the optimization problem as well as the model for inspection
    """
    model = model_factory(items, capacity)
    opt = pyo.SolverFactory(solver)

    solution_state = opt.solve(model)

    solution = Selection([])
    for item in items:
        if round(pyo.value(model.files[item.name])) == 1:
            solution.items.append(item)

    return solution, model


There are lots of solvers to choose from but most of them must be installed on the system before use. By running
`pyomo help` we can see which solvers pyomo supports and which are available on the system.

Pyomo is a general optimization framework and is not restricted to integer programs like this one. It is therefore
important to choose a solver that works well for the problem.

In [ ]:
!pyomo help -s

In [ ]:
solution_ipopt, model_ipopt = solve_pyomo(directories, 1024, 'ipopt')
print(solution_ipopt)

In [ ]:
model_ipopt.pprint()

In [ ]:
solution_cbc, model_cbc = solve_pyomo(directories, 1024, 'cbc')
print(solution_cbc)

In [ ]:
model_cbc.pprint()

In [ ]:
solution_glpk, _ = solve_pyomo(directories, 1024, 'glpk')
print(solution_glpk)

In [ ]:
solution_files_glpk, _ = solve_pyomo(files, 1024, 'glpk')
print(solution_files_glpk)

## Additional constraints

While you were telling your friend about your plan to back up your files on the cloud they got quite worried that you
haven't considered the privacy issues you could potentially face doing this. How do you know that the company running
this new and shiny can be trusted? Skimming their EULA it all seems good, but there is simply too much at risk.
Ideally you should encrypt your information before uploading it to the cloud, but as this isn't a tutorial on
encryption we have no idea how to do that, so we choose the second-best option. We want to limit the amount of
identifying information saved in one location, and we define that the following way

1. We can only have at most two of the following folders backed up
   - `documents`
   - `pictures`
   - `Outlook mailboxes`
2. We can only either back up `documents` or both `university documents` and `talks`

❓ _How do we add these constraints to our model?_
❓ _What about the previous approaches?_

In [ ]:
def create_privacy_model(items: list[Item], capacity: int) -> ConcreteModel:
    """
    Create an optimization model with additional privacy constraints, only works for the list of directories.
    """
    model = create_model(items, capacity)

    model.identifying_information = Constraint(
        expr=model.files['documents'] + model.files['pictures'] + model.files['Outlook mailboxes'] <= 2)
    model.one_set_of_documents = Constraint(
        expr=model.files['documents'] + 0.5 * model.files['university notes'] + 0.5 * model.files['talks'] <= 1)

    return model

In [ ]:
solution_privacy, _ = solve_pyomo(directories, 1024, 'cbc', create_privacy_model)
print('\n', solution_privacy)